# Исследование пользователей и публикаций на StackOverflow

**Цель исследования** - получение различных срезов данных о пользователях и публикациях на StackOverflow с помощью SQL

**Описание данных** \
Анализ проводится на основе базы данных StackOverflow за 2008 год, состоящей из следующих таблиц:
- stackoverflow.badges (информация о значках за достижения)
- stackoverflow.post_types (информация о типе публикаций)
- stackoverflow.posts (информация о публикациях)
- stackoverflow.users (информация о пользователях)
- stackoverflow.vote_types (информация о метках, оставляемых пользователями под публикациями)
- stackoverflow.votes (информация о голосах за публикации)

**План работы**

*Шаг 1 Изучение общей информации*

*Шаг 2 Анализ данных*

- Часть 1 (обучающая)
- Часть 2 (аналитическая)

*Шаг 3 Общий вывод*

## Шаг 1 Изучаем общую информацию

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [3]:
# Автоматизируем вывод 5 первых строк таблиц, отображаем количество строк с помощью небольшой оконной функции при просмотре данных
for table in ['stackoverflow.badges', 'stackoverflow.post_types', 'stackoverflow.posts', 'stackoverflow.users', 'stackoverflow.vote_types', 'stackoverflow.votes']:
    print('Вывод таблицы: ', table)
    display(pd.io.sql.read_sql('''SELECT *, COUNT(*) OVER() AS table_size FROM {} LIMIT 5'''.format(table), con = engine))

Вывод таблицы:  stackoverflow.badges


,id,name,user_id,creation_date,table_size
0,82947,Teacher,994,2008-09-15 08:55:04,330130
1,82949,Teacher,3893,2008-09-15 08:55:04,330130
2,82950,Teacher,4591,2008-09-15 08:55:04,330130
3,82951,Teacher,5196,2008-09-15 08:55:04,330130
4,82952,Teacher,2635,2008-09-15 08:55:04,330130


Вывод таблицы:  stackoverflow.post_types


,id,type,table_size
0,1,Question,2
1,2,Answer,2


Вывод таблицы:  stackoverflow.posts


...

,id,creation_date,display_name,last_access_date,location,reputation,views,table_size
0,1,2008-07-31 14:22:31,Jeff Atwood,2018-08-29 02:34:23,"El Cerrito, CA",44300,408587,24044
1,2,2008-07-31 14:22:31,Geoff Dalgas,2018-08-23 17:31:56,"Corvallis, OR",3491,23966,24044
2,3,2008-07-31 14:22:31,Jarrod Dixon,2018-08-30 20:56:24,"Raleigh, NC, United States",13418,24396,24044
3,4,2008-07-31 14:22:31,Joel Spolsky,2018-08-14 22:18:15,"New York, NY",28768,73755,24044
4,5,2008-07-31 14:22:31,Jon Galloway,2018-08-29 16:48:36,"San Diego, CA",39172,11700,24044


Вывод таблицы:  stackoverflow.vote_types


,id,name,table_size
0,1,AcceptedByOriginator,15
1,2,UpMod,15
2,3,DownMod,15
3,4,Offensive,15
4,5,Favorite,15


Вывод таблицы:  stackoverflow.votes


,id,post_id,user_id,bounty_amount,vote_type_id,creation_date,table_size
0,434364,146794,4918,None,5,2008-10-03,58790
1,434366,135535,16511,None,3,2008-10-03,58790
2,434372,162181,12818,None,1,2008-10-03,58790
3,434380,21965,4002,None,1,2008-10-03,58790
4,434401,165975,8161,None,2,2008-10-03,58790


**Промежуточный вывод:** Анализируемая база данных StackOverflow включает в себя 6 таблиц, содержащих информацию о 24'044 пользователях, 243'796 публикациях двух типов ("вопрос", "ответ"), 58'790 голосах, оставленных за публикации (голоса представлены 15 типами) и 330'130 значках, полученных за достижения.

## Шаг 2 Анализ данных

### Часть 1 (обучающая)

#### **Задача 1** 
Вывести количество вопросов, набравших более 300 очков или добавленных в «Закладки» минимум 100 раз

In [4]:
query = '''
           SELECT COUNT(p.id) AS question_cnt
           FROM stackoverflow.posts AS p 
           JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
           WHERE pt.type = 'Question' 
           AND (p.score > 300 OR p.favorites_count >= 100)
        '''
pd.io.sql.read_sql(query, con = engine)

,question_cnt
0,1355


#### **Задача 2**
Рассчитать среднее количество заданных вопросов в день с 1 по 18 ноября 2008 (результат округляется до целого числа)

In [5]:
query = '''
           SELECT CAST(ROUND(AVG(count), 0) AS int) AS avg_question_cnt
           FROM (SELECT CAST(p.creation_date AS date),
                        COUNT(p.id)
                 FROM stackoverflow.posts AS p 
                 JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
                 WHERE pt.type = 'Question' 
                 AND CAST(p.creation_date AS date) BETWEEN '2008-11-01' AND '2008-11-18' 
                 GROUP BY CAST(p.creation_date AS date)) AS q_cnt  
        '''
pd.io.sql.read_sql(query, con = engine)

,avg_question_cnt
0,383


#### **Задача 3**
Вывести количество уникальных пользователей, получивших значки в день регистрации

In [6]:
query = '''
           SELECT COUNT(DISTINCT u.id) AS user_cnt
           FROM stackoverflow.users AS u 
           JOIN stackoverflow.badges AS b ON u.id=b.user_id 
           WHERE CAST(u.creation_date AS date) = CAST(b.creation_date AS date)
        '''
pd.io.sql.read_sql(query, con = engine)

,user_cnt
0,7047


#### **Задача 4**
Вывести количество уникальных постов пользователя Joel Coehoorn, получивших хотя бы один голос

In [7]:
query = ''' 
           WITH votes_count AS(
                               SELECT p.id AS id_post,
                                      COUNT(v.id) AS votes_cnt
                               FROM stackoverflow.users AS u 
                               JOIN stackoverflow.posts AS p ON u.id=p.user_id
                               JOIN stackoverflow.votes AS v ON p.id=v.post_id
                               WHERE u.display_name = 'Joel Coehoorn' 
                               GROUP BY p.id
                               )

           SELECT COUNT(id_post) AS post_cnt
           FROM votes_count
           WHERE votes_cnt>0
        '''
pd.io.sql.read_sql(query, con = engine)

,post_cnt
0,12


#### **Задача 5**
Выгрузить таблицу vote_types c добавленным полем rank, в которое входят номера записей в обратном порядке (таблицу отсортировать по id)

In [8]:
query = ''' 
           SELECT *,  
                  ROW_NUMBER() OVER (ORDER BY id DESC)
           FROM stackoverflow.vote_types
           ORDER BY id
        '''
pd.io.sql.read_sql(query, con = engine)

,id,name,row_number
0,1,AcceptedByOriginator,15
1,2,UpMod,14
2,3,DownMod,13
3,4,Offensive,12
4,5,Favorite,11
5,6,Close,10
6,7,Reopen,9
7,8,BountyStart,8
8,9,BountyClose,7
9,10,Deletion,6


#### **Задача 6**
Выделить 10 пользователей, поставивших больше всего голосов типа Close. Таблица должна включать два поля (id пользователя и количество голосов) и быть отсортирована по убыванию количества голосов и далее по убыванию значения id пользователя

In [9]:
query = '''
           SELECT u.id,
                  COUNT(u.id) AS voice_cnt
           FROM stackoverflow.users AS u
           JOIN stackoverflow.votes AS v ON u.id=v.user_id
           JOIN stackoverflow.vote_types AS vt ON v.vote_type_id=vt.id 
           WHERE vt.name = 'Close'
           GROUP BY u.id
           ORDER BY voice_cnt DESC, id DESC
           LIMIT 10
        '''
pd.io.sql.read_sql(query, con = engine)

,id,voice_cnt
0,20646,36
1,14728,36
2,27163,29
3,41158,24
4,24820,23
5,9345,23
6,3241,23
7,44330,20
8,38426,19
9,19074,19


#### **Задача 7**
Выделить 10 пользователей по количеству значков, полученных 15 ноя - 15 дек 2008 (включительно). Таблица должна включать id пользователя, число значков, место в рейтинге. Пользователям с одинаковым количеством значков присваивается одинаковое место в рейтинге. Записи должны быть отсортированы по количеству значков по убыванию, далее по возрастанию значения id пользователя

In [10]:
query = '''
           SELECT user_id,
                  COUNT(id) AS badges_cnt,
                  DENSE_RANK() OVER (ORDER BY COUNT(id) DESC)
           FROM stackoverflow.badges
           WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
           GROUP BY user_id
           ORDER BY badges_cnt DESC, user_id ASC
           LIMIT 10
        '''
pd.io.sql.read_sql(query, con = engine)

,user_id,badges_cnt,dense_rank
0,22656,149,1
1,34509,45,2
2,1288,40,3
3,5190,31,4
4,13913,30,5
5,893,28,6
6,10661,28,6
7,33213,25,7
8,12950,23,8
9,25222,20,9


#### **Задача 8**
Рассчитать, сколько в среднем очков получает пост каждого пользователя. Таблица должна включать поля заголовок поста, id пользователя, число очков поста, среднее число очков пользователя за пост, округлённое до целого числа. Посты без заголовка и набравшие ноль не учитываются

In [11]:
query = '''
           SELECT title,
                  user_id,
                  score,
                  CAST(ROUND(AVG(score) OVER (PARTITION BY user_id), 0) AS int) AS avg_score_per_post
           FROM stackoverflow.posts
           WHERE title IS NOT NULL AND score != 0
        '''
pd.io.sql.read_sql(query, con = engine)

,title,user_id,score,avg_score_per_post
0,Diagnosing Deadlocks in SQL Server 2005,1,82,573
1,How do I calculate someone's age in C#?,1,1743,573
2,Why doesn't IE7 copy <pre><code> blocks to the...,1,37,573
3,Calculate relative time in C#,1,1348,573
4,Wrapping StopWatch timing with a delegate or l...,1,92,573
...,...,...,...,...
47349,Multi-lingual projects in Visual Studio,3742716,1,1
47350,Change datatype when importing Excel file into...,3930756,5,5
47351,How to host a Silverlight app in a Sharepoint ...,4020932,8,8
47352,Getting counts for a paged SQL search stored p...,5696608,2,2


#### **Задача 9**
Отобразить заголовки постов, написанные пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список

In [12]:
query = '''        
           WITH badge_cnt AS (
                              SELECT user_id,
                                    COUNT(id)
                              FROM stackoverflow.badges 
                              GROUP BY user_id
                              HAVING COUNT(id) > 1000
                              )
                   
           SELECT title
           FROM badge_cnt
           JOIN stackoverflow.posts AS p ON badge_cnt.user_id=p.user_id
           WHERE title IS NOT NULL
        '''
pd.io.sql.read_sql(query, con = engine)

,title
0,What's the strangest corner case you've seen i...
1,What's the hardest or most misunderstood aspec...
2,What are the correct version numbers for C#?
3,Project management to go with GitHub


#### **Задача 10**
Вывести данные о пользователях из Канады, разделив их на три группы по количеству просмотров их профилей: (1) группа 1 - просмотров больше или равно 350, (2) группа 2 - меньше 350, но больше или равно 100, (3) группа 3 - меньше 100. Таблица должна содержать id пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу

In [13]:
query = '''        
           SELECT id, 
                  views,
                  CASE
                     WHEN views >=350 THEN 1
                     WHEN views < 350 AND views >= 100 THEN 2
                     WHEN views < 100 THEN 3
                  END
           FROM stackoverflow.users 
           WHERE location LIKE '%%Canada%%' AND views != 0  
        '''
pd.io.sql.read_sql(query, con = engine)

,id,views,case
0,22,1079,1
1,34,1707,1
2,37,757,1
3,41,174,2
4,42,1063,1
...,...,...,...
832,431315,103,2
833,266220,13,3
834,474548,24,3
835,455635,17,3


#### **Задача 11**
Дополнить предыдущий запрос, отобразив лидеров каждой группы — пользователей с максимальным числом просмотров в своей группе. Таблица должна содержать id пользователя, группу и количество просмотров и быть отсортирована по убыванию просмотров, а затем по возрастанию значения id 

In [14]:
query = '''        
           WITH canada_views AS (
                                 SELECT id, 
                                        views,
                                        CASE
                                            WHEN views >=350 THEN 1
                                            WHEN views < 350 AND views >= 100 THEN 2
                                            WHEN views < 100 THEN 3
                                        END AS category
                                 FROM stackoverflow.users 
                                 WHERE location LIKE '%%Canada%%' AND views != 0 
                                 )

           SELECT id,
                  views, 
                  category
           FROM (SELECT id,
                        views, 
                        category, 
                        MAX(views) OVER (PARTITION BY category ORDER BY views DESC) AS max_views
                 FROM canada_views) AS canada_max
           WHERE views = max_views
           ORDER BY views DESC, id
        '''
pd.io.sql.read_sql(query, con = engine)

,id,views,category
0,3153,21991,1
1,46981,349,2
2,3444,99,3
3,22273,99,3
4,190298,99,3


#### **Задача 12**
Рассчитать ежедневный прирост новых пользователей в ноябре 2008. Таблица должна включать номер дня, число пользователей, зарегистрированных в этот день, сумму пользователей с накоплением

In [15]:
query = '''        
           WITH user_cnt AS (
                             SELECT CAST(EXTRACT (DAY FROM creation_date) AS int) AS day_number,
                                    COUNT(id) AS user_cnt 
                             FROM stackoverflow.users
                             WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
                             GROUP BY EXTRACT (DAY FROM creation_date)
                             )
                  
           SELECT *,
                  CAST(SUM(user_cnt) OVER (ORDER BY day_number) AS int) AS sum_cum
           FROM user_cnt
        '''
pd.io.sql.read_sql(query, con = engine)

,day_number,user_cnt,sum_cum
0,1,34,34
1,2,48,82
2,3,75,157
3,4,192,349
4,5,122,471
5,6,132,603
6,7,104,707
7,8,42,749
8,9,45,794
9,10,93,887


#### **Задача 13**
Найти интервал между регистрацией и временем создания первого поста для каждого пользователя, написавшего хотя бы один пост. Таблица должна содержать id пользователя, разницу во времени между регистрацией и первым постом

In [16]:
query = '''        
           SELECT DISTINCT p.user_id,
                  MIN(p.creation_date) OVER (PARTITION BY user_id) - u.creation_date
           FROM stackoverflow.posts AS p
           JOIN stackoverflow.users AS u ON p.user_id=u.id
        '''
pd.io.sql.read_sql(query, con = engine)

,user_id,?column?
0,27088,22 days 10:32:25
1,4666,4 days 13:51:01
2,43473,0 days 00:00:00
3,761503,0 days 00:00:00
4,9293,8 days 00:39:24
...,...,...
18844,34508,34 days 19:26:31
18845,18334,6 days 14:36:26
18846,4014,42 days 04:46:18
18847,1658,1 days 14:20:32


### Часть 2 (аналитическая)

#### **Задача 1**
Вывести общую сумму просмотров постов, опубликованных за каждый месяц 2008, с сортировкой по убыванию общего количества просмотров. Месяцы с отсутствующими данными необходимо пропустить 

In [17]:
query = '''        
           SELECT DATE_TRUNC('month', creation_date)::date,
                  SUM(views_count)
           FROM stackoverflow.posts
           GROUP BY DATE_TRUNC('month', creation_date)::date
           ORDER BY sum DESC
        '''
pd.io.sql.read_sql(query, con = engine)

,date_trunc,sum
0,2008-09-01,452928568
1,2008-10-01,365400138
2,2008-11-01,221759651
3,2008-12-01,197792841
4,2008-08-01,131367083
5,2008-07-01,669895


**Комментарий:** В базе данных StackOverflow за 2008 остутствует информация о первом полугодии. Наименьшее количество публикаций сделано в летние месяцы (июль, август), что вероятно связано с традиционным снижением бизнес активности и отпускным сезоном или неполнотой данных. Наибольшее количество постов зарегистрировано с сентября по ноябрь, что совпадает с периодом пиковой деловой активности.

#### **Задача 2**
Вывести имена наиболее активных пользователей, оставивших более 100 ответов в первый месяц после регистрации (включая день регистрации). Таблица должна отображать количество уникальных значений user_id для каждого имени пользователя. Результат по полю с именами необходимо отсортировать в лексикографическом порядке. Вопросы, заданные пользователями, не учитываются.

In [18]:
query = '''        
           SELECT u.display_name,
                  COUNT(DISTINCT u.id)
           FROM stackoverflow.users AS u
           JOIN stackoverflow.posts AS p ON u.id=p.user_id
           JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
           WHERE pt.type='Answer'
           AND DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)
           AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
           GROUP BY u.display_name
           HAVING COUNT(pt.type) >100
           ORDER BY u.display_name
        '''
pd.io.sql.read_sql(query, con = engine)

,display_name,count
0,1800 INFORMATION,1
1,Adam Bellaire,1
2,Adam Davis,1
3,Adam Liss,1
4,Alan,8
...,...,...
74,lomaxx,1
75,mattlant,1
76,paxdiablo,1
77,tvanfosson,1


**Комментарий:** 79 пользователей оставили более 100 ответов в первый месяц после регистрации. Отмечаем, что распространенным именам Alan, Dan или Chris соответствуют несколько значений user_id. Соответственно, при анализе данных по имени результаты могут быть искажены

#### **Задача 3**
Вывести количество постов за 2008 в разбивке по месяцам. Публикации должны быть сделаны пользователями, зарегистрировавшимися в сентябре 2008 и сделавшими хотя бы один пост в декабре 2008. Таблица должна быть отсортирована по убыванию значения месяца

In [19]:
query = '''        
           SELECT CAST(DATE_TRUNC('month', p.creation_date) AS date) AS month,
                  COUNT(DISTINCT id)
           FROM stackoverflow.posts AS p
           WHERE user_id IN (SELECT DISTINCT u.id
                             FROM stackoverflow.users AS u
                             JOIN stackoverflow.posts AS p ON u.id=p.user_id
                             WHERE u.creation_date :: date BETWEEN '2008-09-01' AND '2008-09-30'
                             AND p.creation_date :: date BETWEEN '2008-12-01' AND '2008-12-31')
           GROUP BY month
           ORDER BY month DESC
        '''
pd.io.sql.read_sql(query, con = engine)

,month,count
0,2008-12-01,17641
1,2008-11-01,18294
2,2008-10-01,27171
3,2008-09-01,24870
4,2008-08-01,32


**Комментарий:** Наибольшее количество публикаций пользователей, зарегистрировавшихся в сентябре 2008 и сделавших хотя бы один пост в декабре 2008, максимально в октябре (27'171) и сентябре (24'870). Эти месяцы также являются лидерами по общему количеству публикаций. Отмечается наличие аномальных значений в финальной таблице: пользователи, зарегистрированные в сентябре, были активны в августе. Вероятно к этому привела ошибка данных

#### **Задача 4**
Создать таблицу, содержащую поля id пользователя, написавшего пост, дату публикации, количество просмотров текущей публикации, сумму просмотров постов автора с накоплением. Данные в таблице должны быть отсортированы по возрастанию id пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста

In [20]:
query = '''        
           SELECT user_id,
                  creation_date,
                  views_count, 
                  SUM(views_count) OVER (PARTITION BY user_id ORDER BY user_id, creation_date)
           FROM stackoverflow.posts
        '''
pd.io.sql.read_sql(query, con = engine)

,user_id,creation_date,views_count,sum
0,1,2008-07-31 23:41:00,480476,480476
1,1,2008-07-31 23:55:38,136033,616509
2,1,2008-07-31 23:56:41,0,616509
3,1,2008-08-04 02:45:08,0,616509
4,1,2008-08-04 04:31:03,0,616509
...,...,...,...,...
243791,5696608,2008-12-23 16:00:37,0,2804
243792,5696608,2008-12-23 17:35:09,0,2804
243793,5696608,2008-12-24 01:02:48,0,2804
243794,5696608,2008-12-30 14:34:45,0,2804


**Комментарий:** Выведена информация о 243'796 публикациях с сортировкой по возрастанию id пользователя и возрастанию даты создания поста при совпадении id пользователя

#### **Задача 5**
Найти среднее количество дней с 1 по 7 декабря 2008 (включительно), когда пользователи взаимодействовали с платформой. Для каждого пользователя необходимо отобрать дни, когда он/она опубликовали хотя бы 1 пост. Нужно получить одно целое число с помощью округления

In [21]:
query = '''        
           WITH users_cnt AS( 
                             SELECT user_id,
                                    COUNT(DISTINCT creation_date::date)
                             FROM stackoverflow.posts
                             WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
                             GROUP BY user_id
                             )
           SELECT CAST(ROUND(AVG(count), 0) AS int) AS avg_num_days
           FROM users_cnt
        '''
pd.io.sql.read_sql(query, con = engine)

,avg_num_days
0,2


**Комментарий:** В среднем 2 дня с 1 по 7 декабря 2008 (включительно) пользователи взаимодействовали с платформой

#### **Задача 6**
Рассчитать, на сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008. В таблице необходимо отразить номер месяца, количество постов в месяц, процент изменения количества постов в текущем месяце по сравнению с предыдущим (количество постов в сентябре нужно сравнить со значением предыдущего месяца). Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Необходимо округлить значение процента до двух знаков после запятой. 

*При делении одного целого числа на другое в PostgreSQL в результате получается целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, необходимо перевести делимое в тип numeric*

In [22]:
query = '''        
           WITH posts_cnt AS( 
                             SELECT CAST (EXTRACT (MONTH FROM creation_date) AS int) AS month, 
                                    COUNT(id)
                             FROM stackoverflow.posts 
                             WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
                             GROUP BY month)

           SELECT *,
                  ROUND((count - LAG(count) OVER ())::numeric / LAG(count) OVER ()*100,2) AS perc_difference
           FROM posts_cnt
        '''
pd.io.sql.read_sql(query, con = engine)

,month,count,perc_difference
0,9,70371,NaN
1,10,63102,-10.33
2,11,46975,-25.56
3,12,44592,-5.07


**Комментарий:** В абсолютных значениях максимальное количество публикаций сделано в сентябре, далее до конца года отмечается плавное ежемесячное снижение количества постов. При этом наибольшее сокращение общего количества публикаций наблюдается в ноябре относительно октября (25,56%), наименьшее - в декабре относительно ноября (5,07%). 

#### **Задача 7**
Определить пользователя, опубликовавшего максимальное количество постов за все время с момента публикации. Необходимо вывести данные его активности за октябрь 2008: номер недели, дату и время последнего поста, опубликованного на этой неделе

In [23]:
query = '''        
           WITH oct_activity AS (
                                 SELECT CAST(EXTRACT(WEEK FROM creation_date) AS int) AS week_number,
                                        MAX(creation_date) OVER (PARTITION BY EXTRACT(WEEK FROM creation_date)) AS last_post
                                 FROM stackoverflow.posts
                                 WHERE user_id = (SELECT user_id
                                                  FROM stackoverflow.posts
                                                  GROUP BY user_id
                                                  ORDER BY COUNT(id) DESC
                                                  LIMIT 1)
                                 AND creation_date::date BETWEEN '2008-10-01' AND '2008-10-31'
                                 ORDER BY creation_date
                                 )
           SELECT DISTINCT *
           FROM oct_activity
           ORDER BY week_number
        '''
pd.io.sql.read_sql(query, con = engine)

,week_number,last_post
0,40,2008-10-05 09:00:58
1,41,2008-10-12 21:22:23
2,42,2008-10-19 06:49:30
3,43,2008-10-26 21:44:36
4,44,2008-10-31 22:16:01


**Комментарий:** Определен пользователь с максимальным количеством публикаций за весь период с момента публикации, выведены даты и время его последних публикаций на каждой неделе октября

## Шаг 3 Общий вывод

**Цель исследования** - получение различных срезов данных о пользователях и публикациях на StackOverflow за 2008 год.

**Описание данных**
Анализируемая база данных StackOverflow включает в себя 6 таблиц, содержащих информацию о 24'044 пользователях, 243'796 публикациях двух типов ("вопрос", "ответ"), 58'790 голосах, оставленных за публикации (голоса представлены 15 типами) и 330'130 значках, полученных за достижения.

**Анализ данных**
- Максимальное количество постов зарегистрировано с сентября по ноябрь, что совпадает с периодом пиковой деловой активности. Наименьшее количество публикаций - в летние месяцы (июль, август), что вероятно связано с традиционным снижением бизнес активности и отпускным сезоном или же неполнотой данных. В базе данных StackOverflow за 2008 остутствует информация о первом полугодии.  
- В абсолютных значениях максимальное количество публикаций сделано в сентябре, далее до конца года отмечается плавное ежемесячное снижение количества постов. При этом наибольшее сокращение общего количества публикаций наблюдается в ноябре относительно октября (25,56%), наименьшее - в декабре относительно ноября (5,07%).
- Наибольшее количество публикаций пользователей, зарегистрировавшихся в сентябре 2008 и сделавших хотя бы один пост в декабре 2008, максимально в октябре (27'171) и сентябре (24'870). Эти месяцы также являются лидерами по общему количеству публикаций. Отмечается наличие аномальных значений в финальной таблице: пользователи, зарегистрированные в сентябре, были активны в августе. Вероятно к этому привела ошибка данных.
- 79 пользователей оставили более 100 ответов в первый месяц после регистрации. Распространенным именам Alan, Dan или Chris соответствуют несколько значений user_id. Соответственно, при анализе данных по имени результаты могут быть искажены
- В среднем 2 дня с 1 по 7 декабря 2008 (включительно) пользователи взаимодействовали с платформой

**Рекомендации** \
Учитывая отсутствие данных за первое полугодие 2008 и наличие аномальных значений, рекомендовано изучить процесс сбора данных на предмет ошибок, не позволяющих обеспечить полноту и корректность анализируемой информации   